**Table of contents**<a id='toc0_'></a>    
- [Self-Localization Using Extended Kalman Filter](#toc1_)    
- [Starting pybullet](#toc2_)    
- [Defining the Sensor Class](#toc3_)    
  - [IMU class](#toc3_1_)    
  - [GPS class](#toc3_2_)    
- [Defining the Extended Kalman Filter Class](#toc4_)    
- [Initial Setup](#toc5_)    
- [Setting Parameters](#toc6_)    
  - [Mobile Robot Parameters](#toc6_1_)    
  - [IMU Parameters](#toc6_2_)    
  - [GPS Parameters](#toc6_3_)    
  - [extended kalman filter Parameters](#toc6_4_)    
- [Self-Localization Using Extended Kalman Filter](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Self-Localization Using Extended Kalman Filter](#toc0_)


In this notebook, we perform self-localization using an extended Kalman filter for a mobile robot.

(For a manual summarizing the functions available in pybullet, please refer to [here](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)

# <a id='toc2_'></a>[Starting pybullet](#toc0_)

In [1]:
import time
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[Defining the Sensor Class](#toc0_)

## <a id='toc3_1_'></a>[IMU class](#toc0_)

In [2]:
import numpy as np

class IMU:
    def __init__(self, robot_id, sensor_link_idx, init_posture, acc_noise_dist, gyro_noise_dist):
        """
        IMU class
        Parameters
        ----------
        robot_id : int
            Robot ID
        sensor_link_idx : int
            Link index of the IMU attachment position
        init_posture : list
            Initial posture (roll, pitch, yaw)
        acc_noise_dist : float
            Distribution of accelerometer noise
        gyro_noise_dist : float
            Distribution of gyroscope noise
        """
        self.robot_id = robot_id
        self.sensor_link_idx = sensor_link_idx
        self.posture = np.array(init_posture)
        self.acc_noise_dist = acc_noise_dist
        self.gyro_noise_dist = gyro_noise_dist
        self.acc = np.zeros(3)
        self.gyro = np.zeros(3)
        self.velocity_prev = np.zeros(3)

    def update_acc(self, dt):
        """
        Update acceleration
        """
        # Get link velocity (in pybullet, there is no function to directly obtain acceleration, so velocity is obtained and acceleration is calculated from the time difference)
        link_state = pybullet.getLinkState(self.robot_id, self.sensor_link_idx, computeLinkVelocity=1, computeForwardKinematics=1)
        velocity = np.array(link_state[6])
        self.acc = (velocity - self.velocity_prev) / dt
        self.velocity_prev = velocity

        # Add noise
        self.acc += np.random.normal(0, self.acc_noise_dist, 3)

    def get_acc(self):
        """
        Get acceleration
        """
        return self.acc
    
    def update_gyro(self):
        """
        Update angular velocity
        """
        # Get link angular velocity
        link_state = pybullet.getLinkState(self.robot_id, self.sensor_link_idx, computeLinkVelocity=1, computeForwardKinematics=1)  
        self.gyro = np.array(link_state[7])

        # Add noise
        self.gyro += np.random.normal(0, self.gyro_noise_dist, 3)

    def get_gyro(self):
        """
        Get angular velocity
        """
        return self.gyro

    def update_posture(self, dt):
        """
        Update posture
        """
        # Get robot posture
        self.update_gyro()
        gyro = self.get_gyro()
        self.posture += gyro * dt

        # Keep posture within the range of -π to π
        self.posture = np.mod(self.posture + np.pi, 2*np.pi) - np.pi

    def get_posture(self):
        """
        Get posture
        """
        return self.posture

## <a id='toc3_2_'></a>[GPS class](#toc0_)

In [ ]:
class GPS:
    def __init__(self, robot_id, sensor_link_idx, noise_dist):
        """
        GPS class
        Parameters
        ----------
        robot_id : int
            Robot ID
        sensor_link_idx : int
            Link index of the GPS attachment position
        noise_dist : float
            Distribution of noise
        """
        self.robot_id = robot_id
        self.sensor_link_idx = sensor_link_idx
        self.noise_dist = noise_dist
        self.position = np.zeros(3)

    def update_position(self):
        """
        Update GPS position
        """
        # Get link position
        link_state = pybullet.getLinkState(self.robot_id, self.sensor_link_idx, computeLinkVelocity=1, computeForwardKinematics=1)
        self.position = np.array(link_state[0])[:2]

        # Add noise
        self.position += np.random.normal(0, self.noise_dist, 2)

    def get_position(self):
        """
        Get GPS position
        """
        return self.position

# <a id='toc4_'></a>[Defining the Extended Kalman Filter Class](#toc0_)

The Extended Kalman Filter (EKF) is a method for estimating the state by combining the "state predicted by the motion model" and the "observations obtained from sensors" in a good ratio. The class for the Extended Kalman Filter is shown below.

The general flow is as follows:
1. Initialization
    - In the `__init__` method, the initial state is set.
2. Prediction
    - In the `predict` method, the "state `x_pred` predicted based on the motion model" is calculated.
3. Update
    - In the `update` method, the "state `x_pred` predicted by the `predict` method" and the "observations `z` obtained from sensors" are combined to calculate the "state `x_est` estimated by the information from the motion model and the observations".
    - At this time, the "Kalman gain `K`" is an important parameter that determines the ratio at which the information from the motion model and the information from the observations are combined.

<br>

In summary:
- Estimate the state using the motion model
- → Obtain the state from sensors
- → Combine the "state estimated by the motion model" and the "state obtained from sensors" using the Kalman gain to estimate the final state

This is the flow.


In [4]:
class EKFkalmanFilter:
    def __init__(self, init_x, init_P, Q, R, mu):
        """
        Kalman filter class for a two-wheeled robot in a 2D plane
        Parameters
        ----------
        init_x : numpy.ndarray
            Initial state [x, y, theta]
        init_P : numpy.ndarray
            Initial error covariance matrix
        Q : numpy.ndarray
            Process noise covariance matrix
        R : dict
            Observation noise covariance matrix
        mu : float
            Friction coefficient
        """
        self.x = init_x
        self.P = init_P
        self.Q = Q
        if "wheel_odom" or "imu"  or "gps" in R.keys():
            self.R = R
        else:
            raise ValueError("R_dict must have 'imu' or 'gps' key")
        self.mu = mu
        
    def predict(self, x, u, dt, P_est):
        """
        Perform the prediction step (prediction by motion model)

        Parameters
        ----------
        x : numpy.ndarray
            Current state [x, y, theta]
        u : numpy.ndarray
            Input [v, w]
        dt : float
            Time step
        P_est : numpy.ndarray
            Current error covariance matrix

        Returns
        -------
        x_pred : numpy.ndarray
            State estimated by the motion model [x, y, theta]
        """
        x_pred = self.motion_model(x, u, dt)
        jF = self.jacobian_F(x_pred, u, dt)
        P_pred = jF @ P_est @ jF.T + self.Q
        return x_pred, P_pred

    def motion_model(self, x, u, dt):
        """
        Calculate the state x_pred from the current state x and input u using the motion model of a two-wheeled robot in a 2D plane
        
        Parameters
        ----------
        x : numpy.ndarray
            Current state [x, y, theta]
        u : numpy.ndarray
            Input [v, w]
        dt : float
            Time step

        Returns
        -------
        x_pred : numpy.ndarray
            State estimated by the motion model [x, y, theta]
        """
        theta = x[2]
        F = np.array([[1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0]])
        B = np.array([[dt * np.cos(theta), 0.0],
                      [dt * np.sin(theta), 0.0],
                      [0.0, dt]])
        # x_pred = F @ x + B @ u # Motion model without considering friction

        # Motion model considering friction
        v_friction = u[0] * self.mu
        w_friction = -u[1] * self.mu
        u_friction = np.array([v_friction, w_friction])
        x_pred = F @ x + B @ u_friction
        return x_pred

    def jacobian_F(self, x, u, dt):
        """
        Calculate the Jacobian matrix of the motion model for a two-wheeled robot in a 2D plane
        
        Parameters
        ----------
        x : numpy.ndarray
            State [x, y, theta]
        u : numpy.ndarray
            Input [v, w]
        dt : float
            Time step

        Returns
        -------
        jF : numpy.ndarray
            Jacobian matrix of the motion model for a two-wheeled robot in a 2D plane
        """
        theta = x[2]
        jF = np.array([[1.0, 0.0, -u[0] * dt * np.sin(theta)],
                         [0.0, 1.0, u[0] * dt * np.cos(theta)],
                         [0.0, 0.0, 1.0]])
        return jF

    def update(self, x_pred, P_pred, sensor_type, z):
        """
        Perform the update step (update by observation model)

        Parameters
        ----------
        x_pred : numpy.ndarray
            Predicted state
        z : numpy.ndarray
            Observation
        sensor_type : str
            Type of sensor
        P_pred : numpy.ndarray
            Predicted error covariance matrix

        Returns
        -------
        x_est : numpy.ndarray
            Updated state
        P_est : numpy.ndarray
            Updated error covariance matrix
        """

        # Calculate the Jacobian matrix of the observation model
        jH = self.jacobian_H(sensor_type)

        # Calculate the observation predicted by the observation model
        z_pred = self.observation_model(x_pred, sensor_type)

        # Calculate the Kalman gain
        S = jH @ P_pred @ jH.T + self.R[sensor_type]
        K = P_pred @ jH.T @ np.linalg.inv(S)

        # Update the state
        x_est = x_pred + K @ (z-z_pred)

        # Update the error covariance matrix
        P_est = (np.eye(3) - K @ jH) @ P_pred
        return x_est, P_est

    def observation_model(self, x_pred, sensor_type):
        """
        Calculate the observation from the predicted state x_pred by the observation model

        Parameters
        ----------
        x_pred : numpy.ndarray
            Predicted state
        sensor_type : str
            Type of sensor
        """
        H = self.jacobian_H(sensor_type)
        z_pred = H @ x_pred
        return z_pred

    def jacobian_H(self, sensor_type, z=None):
        """
        Calculate the Jacobian matrix of the observation model
        Parameters
        ----------
        sensor_type : str
            Type of sensor
        z : numpy.ndarray
            Observation (not used in this case as the Jacobian matrix is fixed, but it can be dynamically calculated based on the observation value depending on the adopted observation)
        
        Returns
        -------
        jH : numpy.ndarray
            Jacobian matrix of the observation model
        """
        if sensor_type == "imu":
            jH = np.array([[0.0, 0.0, 1.0]])
        elif sensor_type == "gps":
            jH = np.array([[1.0, 0.0, 0.0],
                           [0.0, 1.0, 0.0]])
        return jH

    def estimation(self, u, z_dict, dt):
        """
        Estimation step of the Kalman filter
        
        Parameters
        ----------
        u : numpy.ndarray
            Input [v, w]
        z_dict : dict
            Dictionary of observations {sensor type: observation}
        dt : float
            Time step

        Returns
        -------
        x_est : numpy.ndarray
            Estimated state [x, y, theta]
        """
        # Prediction step
        x_pred, P_pred = self.predict(self.x, u, dt, self.P)

        # Update step
        for sensor_type, z in z_dict.items():
            x_est, P_est = self.update(x_pred, P_pred, sensor_type, z)
            x_pred = x_est
            P_pred = P_est
        self.x = x_est
        self.P = P_est

        # Keep theta within the range -π to π
        if self.x[2] > np.pi:
            self.x[2] -= 2 * np.pi
        elif self.x[2] < -np.pi:
            self.x[2] += 2 * np.pi

    def get_position(self):
        """
        Get the current state
        Returns
        -------
        x : float
            Current x coordinate
        y : float
            Current y coordinate
        theta : float
            Current angle
        """
        return self.x[0], self.x[1], self.x[2]

ven = Mesa


# <a id='toc5_'></a>[Initial Setup](#toc0_)

In [5]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step)

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

# Load the robot
car_start_pos = [0, 0, 0.1]  # Set initial position (x, y, z)
car_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # Set initial orientation (roll, pitch, yaw)
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf", car_start_pos, car_start_orientation)

# Set camera position for GUI mode
camera_distance = 7.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa


# <a id='toc6_'></a>[Setting Parameters](#toc0_)

Set each parameter.

Changing the standard deviation of the noise will affect the accuracy of the extended Kalman filter.

## <a id='toc6_1_'></a>[Mobile Robot Parameters](#toc0_)

In [6]:
# Joint indices of the left and right wheels
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1

# Constants related to the wheels
WHEEL_THREAD = 0.325 # Distance between wheels (match the distance between wheels in "simple_two_wheel_car.urdf")

# Noise in input u (the accuracy of the position estimation by the extended Kalman filter changes depending on the value)
V_NOISE = 0.5 # Noise in translational velocity
W_NOISE = np.deg2rad(5.0) # Noise in rotational velocity
U_NOISE = np.array([V_NOISE, W_NOISE])

# Create sliders for translational velocity and rotational velocity
linear_vel_slider = pybullet.addUserDebugParameter("linear_velocity", -10, 10, 0)
angular_vel_slider = pybullet.addUserDebugParameter("angular_velocity", -10, 10, 0)

## <a id='toc6_2_'></a>[IMU Parameters](#toc0_)

In [7]:
IMU_LINK_IDX = 5
ACC_NOISE_DIST = 0.1 # Standard deviation of the noise in the accelerometer (the accelerometer information is not used in the current extended Kalman filter estimation)
GYRO_NOISE_DIST = np.deg2rad(5.0) # Standard deviation of the noise in the gyroscope (the value affects the accuracy of the position estimation by the extended Kalman filter)

## <a id='toc6_3_'></a>[GPS Parameters](#toc0_)

In [8]:
GPS_LINK_IDX = 5
GPS_NOISE_DIST = 1.0 # Standard deviation of the noise in the GPS (the value affects the accuracy of the position estimation by the extended Kalman filter)

## <a id='toc6_4_'></a>[extended kalman filter Parameters](#toc0_)

In [9]:
# Process noise covariance matrix (specifically considering noise due to friction and "input noise U_NOISE". The closer it is to the true noise, the better the estimation accuracy)
# * Since it is often unknown how much noise actually occurs, it is often set through trial and error
Q = np.array([[0.1, 0.0, 0.0], # Standard deviation of noise in the x direction
              [0.0, 0.1, 0.0], # Standard deviation of noise in the y direction
              [0.0, 0.0, np.deg2rad(1.0)] # Standard deviation of noise in the angle
            ]) ** 2

# Observation noise covariance matrix (specifically considering "IMU noise GYRO_NOISE_DIST" and "GPS noise GPS_NOISE_DIST". The closer it is to the true noise, the better the estimation accuracy)
# * Since it is often unknown how much noise actually occurs, it is often set through trial and error
R = {"imu": np.array([[np.deg2rad(5.0)]]),  # Standard deviation of noise in the yaw angle of the IMU
    "gps": np.array([[1.0, 0.0], # Standard deviation of noise in the x direction of the GPS
                     [0.0, 1.0]]) # Standard deviation of noise in the y direction of the GPS
   }

MU = 0.05 # Friction coefficient (a coefficient to consider friction in the motion model)

# Generate spherical objects to visualize the estimated position (position estimation by motion model only is red, position estimation by extended Kalman filter is green)
motion_model_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=0.1, rgbaColor=[1,0,0,1])
motion_model_id = pybullet.createMultiBody(0, -1, motion_model_visual_id, [0.0, 0.0, 0.0], useMaximalCoordinates=True)
ekf_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=0.1, rgbaColor=[0,1,0,1])
ekf_id = pybullet.createMultiBody(0, -1, ekf_visual_id, [0.0, 0.0, 0.0], useMaximalCoordinates=True)

# <a id='toc7_'></a>[Self-Localization Using Extended Kalman Filter](#toc0_)

When the simulation is executed, the robot's self-position is estimated using the extended Kalman filter.

Additionally, the "true position of the robot," the "position estimated only by the motion model," and the "position estimated by the extended Kalman filter" are displayed on the screen.

In [11]:
import sys
import random

# Initialize debug drawing
pybullet.removeAllUserDebugItems()

# Set the robot to the initial position
car_start_x = 0.0
car_start_y = 5.0
car_start_theta = 0.0
car_start_pos = [car_start_x, car_start_y, car_start_theta]
car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, car_start_theta])
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)

# Instantiate the IMU class
imu_init_posture = [car_start_x, car_start_y, car_start_theta]
imu = IMU(car_id, IMU_LINK_IDX, imu_init_posture, ACC_NOISE_DIST, GYRO_NOISE_DIST)

# Instantiate the GPS class
gps = GPS(car_id, GPS_LINK_IDX, GPS_NOISE_DIST)

# Settings for the Extended Kalman Filter ===========================================================
# Initial values
init_x = np.array([car_start_x, car_start_y, car_start_theta]) # Initial state
init_P = np.array([[0.1, 0.0, 0.0], # Initial error covariance in the x direction
                   [0.0, 0.1, 0.0], # Initial error covariance in the y direction
                   [0.0, 0.0, np.deg2rad(1.0)]]) ** 2 # Initial error covariance for the angle
ekf = EKFkalmanFilter(init_x, init_P, Q, R, MU)
# =============================================================================================

# Variables for position estimation using only the motion model (for comparison with the Extended Kalman Filter estimates)
motion_model_x = car_start_x
motion_model_y = car_start_y
motion_model_theta = car_start_theta

# Settings to display the estimated position on the Pybullet screen (may be heavy)
enable_debug_text = False
replace_debug_text_unique_ids = []

# Control the mobile robot according to the values set on the sliders and calculate the odometry
while True:
    # Get the values from the sliders
    linear_velocity = pybullet.readUserDebugParameter(0)
    angular_velocity = pybullet.readUserDebugParameter(1)

    # Add noise to the input values
    linear_velocity_noised = linear_velocity + np.random.normal(0, U_NOISE[0], 1)
    angular_velocity_noised = angular_velocity + np.random.normal(0, U_NOISE[1], 1)

    # Calculate the speed of the left and right wheels from the translational and rotational speeds (with added noise)
    right_wheel_velocity = linear_velocity_noised - angular_velocity_noised * WHEEL_THREAD / 2
    left_wheel_velocity = linear_velocity_noised + angular_velocity_noised * WHEEL_THREAD / 2

    # Set the speed
    if linear_velocity == 0.0 and angular_velocity == 0.0:
        pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)
        pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=0)
    else:
        pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
        pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)
    pybullet.stepSimulation()

    # Get the sensor values from the IMU
    imu.update_posture(time_step)
    imu_posture = imu.get_posture()
    imu_yaw = np.array(imu_posture[2])

    # Get the sensor values from the GPS
    gps.update_position()
    gps_pos = gps.get_position()

    # Extended Kalman Filter processing ===========================================================
    # Estimation step of the Extended Kalman Filter
    u = np.array([linear_velocity, angular_velocity]) # Input (the input given to the Extended Kalman Filter does not include noise)
    z_dict = {"imu": imu_yaw, "gps": gps_pos}
    ekf.estimation(u, z_dict, time_step)
    ekf_x, ekf_y, ekf_theta = ekf.get_position()
    # =============================================================================================

    # True position, position estimated by odometry, position estimated by the Extended Kalman Filter
    true_car_pos, true_car_orientation = pybullet.getBasePositionAndOrientation(car_id)
    true_x = true_car_pos[0]
    true_y = true_car_pos[1]
    true_euler = pybullet.getEulerFromQuaternion(true_car_orientation)
    true_theta = true_euler[2]

    # Position estimated by the motion model only
    motion_model_pos = ekf.motion_model(np.array([motion_model_x, motion_model_y, motion_model_theta]), u, time_step)
    motion_model_x = motion_model_pos[0]
    motion_model_y = motion_model_pos[1]
    motion_model_theta = motion_model_pos[2]
    if motion_model_theta > np.pi:
        motion_model_theta -= 2 * np.pi
    elif motion_model_theta < -np.pi:
        motion_model_theta += 2 * np.pi

    # Move the spherical object to the estimated position
    pybullet.resetBasePositionAndOrientation(motion_model_id, [motion_model_x, motion_model_y, 0.0], pybullet.getQuaternionFromEuler([0.0, 0.0, motion_model_theta]))
    pybullet.resetBasePositionAndOrientation(ekf_id, [ekf_x, ekf_y, 0.0], pybullet.getQuaternionFromEuler([0.0, 0.0, ekf_theta]))

    # Display "True position", "Position estimation by motion model only", "Position estimation by Extended Kalman Filter"
    # When the movement is small, "Position accuracy by motion model only ≒ Position accuracy by EKF" (e.g., linear_velocity=3.0, angular_velocity=10.0, etc.)
    # As the movement becomes larger, "Position accuracy by motion model only < Position accuracy by EKF" (e.g., linear_velocity=10.0, angular_velocity=3.0, etc.)
    sys.stdout.write("\rTrue: ({:.2f}, {:.2f}, {:.2f}), motion model: ({:.2f}, {:.2f}, {:.2f}), EKF: ({:.2f}, {:.2f}, {:.2f})                   ".format(true_x, true_y, true_theta, motion_model_x, motion_model_y, motion_model_theta, ekf_x, ekf_y, ekf_theta))
    if enable_debug_text:
        if len(replace_debug_text_unique_ids) == 0:
            replace_debug_text_unique_ids.append(pybullet.addUserDebugText("True: ({:.2f}, {:.2f}, {:.2f})".format(true_x, true_y, true_theta), [-2.0, -0.5, 0.0], textSize=2, textColorRGB=[0,0,0]))
            replace_debug_text_unique_ids.append(pybullet.addUserDebugText("Motion model: ({:.2f}, {:.2f}, {:.2f})".format(motion_model_x, motion_model_y, motion_model_theta), [-2.0, -0.9, 0.0], textSize=2, textColorRGB=[0,0,0]))
            replace_debug_text_unique_ids.append(pybullet.addUserDebugText("EKF: ({:.2f}, {:.2f}, {:.2f})".format(ekf_x, ekf_y, ekf_theta), [-2.0, -1.3, 0.0], textSize=2, textColorRGB=[0,0,0]))
        else:
            pybullet.addUserDebugText("True: ({:.2f}, {:.2f}, {:.2f})".format(true_x, true_y, true_theta), [-2.0, -0.5, 0.0], replaceItemUniqueId=replace_debug_text_unique_ids[0], textSize=2, textColorRGB=[0,0,0])
            pybullet.addUserDebugText("Motion model: ({:.2f}, {:.2f}, {:.2f})".format(motion_model_x, motion_model_y, motion_model_theta), [-2.0, -0.9, 0.0], replaceItemUniqueId=replace_debug_text_unique_ids[1], textSize=2, textColorRGB=[0,0,0])
            pybullet.addUserDebugText("EKF: ({:.2f}, {:.2f}, {:.2f})".format(ekf_x, ekf_y, ekf_theta), [-2.0, -1.3, 0.0], replaceItemUniqueId=replace_debug_text_unique_ids[2], textSize=2, textColorRGB=[0,0,0])

True: (-2.37, 4.20, 0.60), motion model: (-0.22, 4.99, 0.06), EKF: (-2.39, 4.27, 0.68)                        

KeyboardInterrupt: 